<a href="https://colab.research.google.com/github/mai0elkady/Disaster_response_pipelines/blob/main/notebooks/ML_Pipeline_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import sklearn
print(sklearn.__version__)

0.22.2.post1


In [ ]:
# load data from database
engine = create_engine('sqlite:////content/drive/MyDrive/UdacityND/project2/data/disaster_data.db')
df = pd.read_sql_table('messages_categories',engine)

X = df['message'].values
Y = df.iloc[:,4:]

In [ ]:
X.shape

(26216,)

In [ ]:
Y.shape

(26216, 35)

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    words = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in words:
        if tok not in stopwords.words("english"):
            clean_tok = lemmatizer.lemmatize(tok).lower().strip()
            clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
# build pipeline
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state = 0)))
])


In [ ]:
pipeline.get_params()

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
# train classifier
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 42)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [ ]:
print(y_pred)

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [ ]:
print(classification_report(y_test, y_pred,target_names=y_test.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      4944
               request       0.84      0.47      0.60      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.76      0.68      0.72      2670
          medical_help       0.72      0.10      0.18       535
      medical_products       0.85      0.06      0.12       344
     search_and_rescue       0.63      0.08      0.13       159
              security       0.25      0.01      0.02       116
              military       0.55      0.06      0.11       200
                 water       0.88      0.27      0.41       418
                  food       0.85      0.54      0.66       745
               shelter       0.84      0.34      0.48       581
              clothing       0.71      0.05      0.10        98
                 money       0.80      0.06      0.11       133
        missing_people       0.50      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
precision_recall_fscore_support(y_test, y_pred, average='micro')

(0.8099908759124088, 0.5194558221181977, 0.6329768270944741, None)

In [ ]:
def eval_results( y_test, y_pred):
    print(classification_report(y_test, y_pred,target_names=y_test.columns.values))
    result = precision_recall_fscore_support(y_test, y_pred, average='micro')
    print(result)
        
eval_results(y_test, y_pred)

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      4944
               request       0.84      0.47      0.60      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.76      0.68      0.72      2670
          medical_help       0.72      0.10      0.18       535
      medical_products       0.85      0.06      0.12       344
     search_and_rescue       0.63      0.08      0.13       159
              security       0.25      0.01      0.02       116
              military       0.55      0.06      0.11       200
                 water       0.88      0.27      0.41       418
                  food       0.85      0.54      0.66       745
               shelter       0.84      0.34      0.48       581
              clothing       0.71      0.05      0.10        98
                 money       0.80      0.06      0.11       133
        missing_people       0.50      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'tfidf__use_idf':[True, False],
              'clf__estimator__criterion': ['gini','entropy'],
              'clf__estimator__n_estimators': [5, 10]}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose = 2, scoring='f1_micro', cv=3) 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train,y_train)
y_pred_new = cv.predict(X_test)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=5, tfidf__use_idf=True 
[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=5, tfidf__use_idf=True, total= 1.5min
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=5, tfidf__use_idf=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s


[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=5, tfidf__use_idf=True, total= 1.4min
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=5, tfidf__use_idf=True 
[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=5, tfidf__use_idf=True, total= 1.4min
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=5, tfidf__use_idf=False 
[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=5, tfidf__use_idf=False, total= 1.4min
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=5, tfidf__use_idf=False 
[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=5, tfidf__use_idf=False, total= 1.4min
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=5, tfidf__use_idf=False 
[CV]  clf__estimator__criterion=gini, clf__estimator__n_estimators=5, tfidf__use_idf=False, total= 1.4min
[CV] clf__estimator__criterion=gini, clf__estimator__n_estimators=10, tfidf__use_idf=True 
[CV]  clf__estimat

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 37.6min finished


TypeError: ignored

In [ ]:
eval_results( y_test, y_pred_new)

                        precision    recall  f1-score   support

               related       0.82      0.91      0.86      4944
               request       0.68      0.44      0.53      1111
                 offer       1.00      0.03      0.06        33
           aid_related       0.68      0.64      0.66      2670
          medical_help       0.58      0.18      0.27       535
      medical_products       0.55      0.12      0.20       344
     search_and_rescue       0.23      0.04      0.07       159
              security       0.08      0.01      0.02       116
              military       0.43      0.10      0.16       200
                 water       0.77      0.48      0.59       418
                  food       0.76      0.41      0.53       745
               shelter       0.72      0.31      0.43       581
              clothing       0.68      0.29      0.40        98
                 money       0.61      0.08      0.15       133
        missing_people       0.29      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(cv.best_params_)
best_model = cv.best_estimator_

{'clf__estimator__criterion': 'gini', 'clf__estimator__n_estimators': 5, 'tfidf__use_idf': True}


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# build pipeline
pipeline_logreg = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LogisticRegression(random_state=0)))
])

pipeline_logreg.get_params()

{'clf': MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                    dual=False,
                                                    fit_intercept=True,
                                                    intercept_scaling=1,
                                                    l1_ratio=None, max_iter=100,
                                                    multi_class='auto',
                                                    n_jobs=None, penalty='l2',
                                                    random_state=0,
                                                    solver='lbfgs', tol=0.0001,
                                                    verbose=0,
                                                    warm_start=False),
                       n_jobs=None),
 'clf__estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max

In [ ]:
pipeline_logreg.fit(X_train,y_train)
y_pred = pipeline_logreg.predict(X_test)
eval_results(y_test, y_pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      4944
               request       0.83      0.53      0.65      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.76      0.67      0.71      2670
          medical_help       0.67      0.16      0.26       535
      medical_products       0.82      0.18      0.30       344
     search_and_rescue       1.00      0.04      0.08       159
              security       0.00      0.00      0.00       116
              military       0.57      0.08      0.14       200
                 water       0.80      0.47      0.59       418
                  food       0.87      0.59      0.70       745
               shelter       0.83      0.48      0.60       581
              clothing       0.88      0.14      0.25        98
                 money       0.63      0.09      0.16       133
        missing_people       1.00      

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
parameters = {'tfidf__use_idf':[True, False],
              'clf__estimator__C': [0.001,10,100],
              }

cv = GridSearchCV(pipeline_logreg, param_grid=parameters, verbose = 2, scoring='f1_micro') 
cv.fit(X_train,y_train)
y_pred_new = cv.predict(X_test)
eval_results(y_test, y_pred_new)

In [ ]:
print(cv.best_params_)
best_model = cv.best_estimator_

{'clf__estimator__C': 10, 'tfidf__use_idf': False}


In [ ]:
# build pipeline
pipeline_SVM = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(SVC(random_state=0)))
])

pipeline_SVM.get_params()

{'clf': MultiOutputClassifier(estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                     class_weight=None, coef0=0.0,
                                     decision_function_shape='ovr', degree=3,
                                     gamma='scale', kernel='rbf', max_iter=-1,
                                     probability=False, random_state=0,
                                     shrinking=True, tol=0.001, verbose=False),
                       n_jobs=None),
 'clf__estimator': SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
     max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
     verbose=False),
 'clf__estimator__C': 1.0,
 'clf__estimator__break_ties': False,
 'clf__estimator__cache_size': 200,
 'clf__estimator__class_weight': None,
 'clf__estimator__coef0': 0.0,
 'clf__estimator__decision_function_shape': 'ovr',
 'clf__est

In [ ]:
parameters = {'tfidf__use_idf':[True, False],
              'clf__estimator__C': [0.001,1,10,100],
              }

cv = GridSearchCV(pipeline_SVM, param_grid=parameters, verbose = 2, scoring='f1_micro') 
cv.fit(X_train,y_train)
y_pred_new = cv.predict(X_test)
eval_results(y_test, y_pred_new)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] clf__estimator__C=0.001, tfidf__use_idf=True ....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... clf__estimator__C=0.001, tfidf__use_idf=True, total= 7.0min
[CV] clf__estimator__C=0.001, tfidf__use_idf=True ....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.0min remaining:    0.0s


[CV] ..... clf__estimator__C=0.001, tfidf__use_idf=True, total= 7.5min
[CV] clf__estimator__C=0.001, tfidf__use_idf=True ....................
[CV] ..... clf__estimator__C=0.001, tfidf__use_idf=True, total= 7.6min
[CV] clf__estimator__C=0.001, tfidf__use_idf=True ....................
[CV] ..... clf__estimator__C=0.001, tfidf__use_idf=True, total= 8.0min
[CV] clf__estimator__C=0.001, tfidf__use_idf=True ....................
[CV] ..... clf__estimator__C=0.001, tfidf__use_idf=True, total= 7.9min
[CV] clf__estimator__C=0.001, tfidf__use_idf=False ...................
[CV] .... clf__estimator__C=0.001, tfidf__use_idf=False, total= 8.2min
[CV] clf__estimator__C=0.001, tfidf__use_idf=False ...................
[CV] .... clf__estimator__C=0.001, tfidf__use_idf=False, total= 8.3min
[CV] clf__estimator__C=0.001, tfidf__use_idf=False ...................
[CV] .... clf__estimator__C=0.001, tfidf__use_idf=False, total= 8.3min
[CV] clf__estimator__C=0.001, tfidf__use_idf=False ...................
[CV] .

### 9. Export your model as a pickle file

In [ ]:
import pickle
pickle.dump(best_model, open("classifier", 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.